In [1]:
import sys

sys.path.append("../../src/stong_core_cython")

In [2]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
import concurrent.futures
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)

from atom import *
from hartree_fock import *
from basis_data import *

In [3]:
df = pd.read_csv("../make_base/sto_3g.csv")

In [4]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 3, 1)}
    hartreefock = HartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [5]:
atom_symbol = "H"
zetas = [1.0, 1.0, 1.0]
atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
molecule = Molecule(atoms)
basis_data = {atom_symbol: create_basis_data(df, zetas, 3, 1)}
print(basis_data)

{'H': {'1s': {'exps': array([0.10998665, 0.40715391, 2.24601403]), 'coefs': array([0.44557585, 0.53536165, 0.15348239]), 'lmn': array([0, 0, 0]), 'zeta': 1.0}, '2s': {'exps': array([0.07475926, 0.22912437, 0.97960552]), 'coefs': array([ 0.69467252,  0.40561594, -0.10023971]), 'lmn': array([0, 0, 0]), 'zeta': 1.0}, '2px': {'exps': array([0.07475926, 0.22912437, 0.97960552]), 'coefs': array([0.38797596, 0.60886703, 0.15855739]), 'lmn': array([1, 0, 0]), 'zeta': 1.0}, '2py': {'exps': array([0.07475926, 0.22912437, 0.97960552]), 'coefs': array([0.38797596, 0.60886703, 0.15855739]), 'lmn': array([0, 1, 0]), 'zeta': 1.0}, '2pz': {'exps': array([0.07475926, 0.22912437, 0.97960552]), 'coefs': array([0.38797596, 0.60886703, 0.15855739]), 'lmn': array([0, 0, 1]), 'zeta': 1.0}, '3s': {'exps': array([0.05586706, 0.14396172, 0.46628257]), 'coefs': array([ 0.96307887,  0.15617196, -0.21512882]), 'lmn': array([0, 0, 0]), 'zeta': 1.0}, '3px': {'exps': array([0.05586706, 0.14396172, 0.46628257]), 'coef

In [6]:
# elements = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne"]
elements = ["H"]
zetas = {}
results = {}
for i, element in enumerate(elements):
    print(element)
    if i == 0:
        result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    else:
        result = minimize(get_energy, result.x, args=(element))
    # result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    results[element] = result
    zetas[element] = result.x
    print(zetas[element])

H
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, p

In [7]:
results

{'H':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -0.49862951086917534
         x: [ 1.787e+00  2.292e+00  1.499e+00]
       nit: 28
       jac: [-3.558e-06 -1.062e-06 -6.694e-06]
  hess_inv: [[ 1.216e+02  1.007e+02  4.375e+01]
             [ 1.007e+02  2.652e+02  9.175e+01]
             [ 4.375e+01  9.175e+01  3.870e+01]]
      nfev: 148
      njev: 37}

In [8]:
df_zetas = pd.DataFrame(zetas)
df_zetas.to_csv("zetas.csv", index=False)